## Week 6 Demo: Inference
#### This demo is adapted from D5.

First, import packages and load our data:

In [ ]:
# Import seaborn and apply its plotting styles
import seaborn as sns
sns.set(font_scale=2, style="white")

import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
# set plotting size parameter
plt.rcParams['figure.figsize'] = (17, 7)

# import pandas & numpy library
import pandas as pd
import numpy as np
import scipy.stats as stats

# Statmodels
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

df_fifa20 = pd.read_csv('players.csv') 

In [ ]:
df_fifa20

## EDA & Visualization

For EDA, let's visualize the numerical columns in our dataframe. Let's take a look at the distribution of the thing we want to predict: value_eur

In [ ]:
value_eur_plot = sns.histplot(df_fifa20['value_eur'], bins = 20)

Let's also take a look at the distribution of `international_reputation`. Visualize `international_reputation` as a bar graph to see the counts of each reputation score.

In [ ]:
int_rep_plot = sns.countplot(data = df_fifa20, x = 'international_reputation')

Now that we’ve looked at individual columns, it’s also useful to explore how different columns relate to one another.
In this dataset, both `overall` and `potential` describe a player’s performance:

`overall` — the player’s current performance rating

`potential` — the player’s projected future performance rating

Let’s visualize how these two metrics relate to each other. We’ll create a plotting matrix (pairplot) to see whether players with higher overall scores also tend to have higher potential.

In [ ]:
potential_vs_overall_plot = sns.pairplot(df_fifa20[['potential', 'overall']])

### Understanding the Relationship Between `overall` and `potential`

From our pairplot visualization, we can see that **`overall`** and **`potential`** have a roughly linear relationship.

In general:
- `potential` represents a player’s **projected** future performance.
- `overall` represents their **current** performance.

The pattern shows that `potential` is usually greater than or equal to `overall`, meaning most players are expected to improve over time.  
If you know a bit about the game, this makes sense — younger players may have great physical skills but less strategic experience, so their current rating (`overall`) is lower than their projected ceiling (`potential`). With experience and training, they can grow closer to that potential.  

Teams may also evaluate players based on `potential`, hoping to **develop** them into top performers rather than buying players who are already at their peak.

---

### Exploring More Relationships

We can also use `sns.pairplot()` to explore **multiple numerical columns** together.  
Let’s look at the relationships between these numerical features:
- `potential`
- `age`
- `international_reputation`
- `value_eur`

This helps us see how factors like age or reputation might relate to a player’s potential and market value.

In [ ]:
num_cols_pair_plot = sns.pairplot(df_fifa20[['potential', 'age', 'international_reputation', 'value_eur']])

### Exploring Trends in the Pairwise Plots

Looking at our pairwise plots, we can see some **interesting patterns** between `international_reputation`, `potential`, `age`, and `value_eur`.

- **`value_eur` vs. `potential`** – The relationship looks **exponential**, meaning that as potential increases, player value increases at an accelerating rate.  
- **`value_eur` vs. `international_reputation`** – Also appears nonlinear; players with higher reputation tend to have **much higher market values**, even with small changes in reputation.  
- **`value_eur` vs. `age`** – Player value often peaks in the **mid-20s**, suggesting a curved (possibly quadratic) relationship.

These nonlinear patterns matter because when we use **Ordinary Least Squares (OLS)** regression, we assume that the **residual errors** (the differences between observed and predicted values) are **normally distributed**.  
OLS does *not* require that the variables themselves are normal — but if the input or target variables are heavily skewed, that skewness can lead to non-normal residuals and unreliable model results.

Since `value_eur` seems to have an **exponential distribution**, let’s check how it looks after applying a **logarithmic transformation**.  
A log transform can make highly skewed variables more symmetric, which may improve model performance and validity.

---

### Visualizing `value_eur` with a Log Transform

We'll plot the distribution of `value_eur` on a **logarithmic scale** to see if it becomes closer to normal.

In [ ]:
value_eur_hist_log = sns.histplot(df_fifa20['value_eur'], log_scale= True)

### Quadratic Transformation of Player Age

Let’s revisit the scatterplot of **`age` vs. `value_eur`** from the pairplot above.  
You may notice a strong **peak** in player value around **26 years old**, with values dropping for both younger and older players.

Here’s some soccer context that helps explain this trend:
- **Younger players** are still developing their technique and decision-making under pressure.  
- **Older players** (past 30) often start to lose speed and agility, reducing their market value.  
- **Players around 25–26** are at the beginning of their *peak performance window*, and teams value them highly because they still have several “prime” years ahead.  

So it makes sense that the curve of player value versus age has an **inverted U-shape**, peaking around 26 years old.

---

### Building an Inverted U-Shaped (Quadratic) Transformation

We can model this pattern mathematically using a quadratic function centered at 26 years old:

$
\text{quad\_age} = -a \cdot (\text{age} - 26)^2 + b
$

where:
- `a` and `b` are constants that determine the *width* and *height* of the curve.
- The negative sign in front ensures it opens **downward** (an inverted U-shape).

We’ll scale this curve from **0 to 1**, purely for convenience.  
To find the constants `a` and `b`:

1. Compute the maximum possible value of $(\text{age} - 26)^2$ from the data.  
   - This occurs for the youngest or oldest players in the dataset.  
2. Set $ a = 1 / \max{((\text{age} - 26)^2)} $ so the curve equals 0 at those extremes.  
3. Set $ b = 1 $ so the curve reaches its peak (value 1) at age 26.

In [ ]:
print('raw age\n', df_fifa20['age'].describe())

In [ ]:
quad_a = 1./(42-26)**2
quad_b = 1.

### Inferential Analysis

### Choosing the Best Model Format for Predicting `value_eur`

In this section, we’ll explore what **model format** gives us the strongest explanatory power for predicting a player’s market value (`value_eur`).  

By “model format,” we mean exploring different **combinations of transformations and predictors**:

- Should we predict **log(`value_eur`)** instead of raw `value_eur`?  
- Should we use **`overall`** or **`potential`** as a performance predictor?  
- Should we use **`age`** or our new **quadratic transformation (`quad_age`)**?  
- Should we include **`international_reputation`**, or leave it out?  
- Should we include a **categorical variable like `nationality`**, or drop it?

---

### 2 × 2 × 2 × 2 × 2 ... Oh No 😱

If we tested every combination of those modeling choices:
- That’s \( 2^5 = 32 \) possible models already.  
- And if we also consider that we might **omit** predictors like `age` or `overall` entirely, that’s actually around **72 models**!  

That’s way too many to fit manually.  
Besides, trying that many combinations would create a **multiple comparison nightmare** — we’d risk **overfitting** and **falsely rejecting null hypotheses** (Type I errors).

---

### A Smarter Way: Forward Model Selection

Instead of brute-forcing all possible models, we can use a variation of **forward model selection**, guided by both data and domain knowledge.

1. **Start with domain knowledge**  
   Identify independent variables that could reasonably affect market value:  
   `overall`, `potential`, `age`, `quad_age`, `international_reputation`, `nationality`  
   And remember — we’ll test both **`value_eur`** and **`log_value_eur`** as our dependent variables (`y`).

2. **Fit simple one-variable models**  
   For every possible **(x, y)** pair, fit a simple model:  
   $
   y \sim x
   $  
   Record each model’s **R-squared** and **p-value**.  
   Models with higher R-squared values explain more variance.

3. **Add predictors one by one**  
   Start with the **best single predictor** (highest R-squared), then add the **next most promising variable**.  
   Keep the new variable **only if**:
   - The **Adjusted R-squared** improves, **and**
   - The new variable’s **p-value < 0.05**

4. **Repeat**  
   Continue adding variables until adding new ones no longer improves the model’s Adjusted R².

In [ ]:
# lets explore all single y ~ x possibilities
df_fifa20['quad_age'] = -quad_a*(df_fifa20['age']-26)**2 + quad_b
df_fifa20['log_value_eur'] = np.log10(df_fifa20['value_eur'] + 1)
df_top4nats = df_fifa20.query('nationality in ["England", "Germany", "Spain", "France"]')

val_overall = smf.ols('value_eur ~ overall',
                   data = df_top4nats
                  ).fit()

log_overall = smf.ols('log_value_eur ~ overall',
                   data = df_top4nats
                  ).fit()

val_potential = smf.ols('value_eur ~ potential',
                   data = df_top4nats
                  ).fit()

log_potential = smf.ols('log_value_eur ~ potential',
                   data = df_top4nats
                  ).fit()

val_age = smf.ols('value_eur ~ age',
                   data = df_top4nats
                  ).fit()

log_age= smf.ols('log_value_eur ~ age',
                   data = df_top4nats
                  ).fit()

val_quad = smf.ols('value_eur ~ quad_age',
                   data = df_top4nats
                  ).fit()

log_quad= smf.ols('log_value_eur ~ quad_age',
                   data = df_top4nats
                  ).fit()

val_intl = smf.ols('value_eur ~ international_reputation',
                   data = df_top4nats
                  ).fit()

log_intl= smf.ols('log_value_eur ~ international_reputation',
                   data = df_top4nats
                  ).fit()

val_nat = smf.ols('value_eur ~ C(nationality)',
                   data = df_top4nats
                  ).fit()

log_nat= smf.ols('log_value_eur ~ C(nationality)',
                   data = df_top4nats
                  ).fit()

singlevar_models = [val_overall, log_overall, 
                    val_potential, log_potential, 
                    val_age, log_age, 
                    val_quad, log_quad,
                    val_intl, log_intl,
                    val_nat, log_nat
                   ]

for model in singlevar_models:
    print(model.summary())


### Q IIIa — Is `log_value_eur` or `value_eur` the Better Thing to Predict?

Now that we’ve fit our simple one-variable models, let’s compare the **R-squared values** for the two dependent variables:

- `value_eur`  
- `log_value_eur`

Look at the printed table of model fits above.  
You should notice that **one of these consistently gives higher R-squared values** across all independent variables.  

That means it explains **more variance** in player value, regardless of which predictor we use.

In [ ]:
best_y = 'log_value_eur'

Hope this helps!